# Plotting and comparing best fit models to data for fits to BAOs with and without the phase shift parameter 

### load stuff 

In [ ]:
# Import some necessary modules
import os
import sys
sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     
from barry.cosmology.camb_generator import Omega_m_z


from barry.samplers import DynestySampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 

# settings 
CV = True

### load in the data 

In [ ]:

datafile="desi_kp4_abacus_cubicbox_pk_elg.pkl"

if CV:
    datafile="desi_kp4_abacus_cubicbox_cv_pk_elg.pkl"


dataset_pk = PowerSpectrum_DESI_KP4(
recon='sym',
fit_poles=[0, 2],
min_k=0.03,
max_k=0.30,
realisation=None,          # realisation=None loads the average of all the realisations
num_mocks=1000,            # Used for Hartlap/Sellentin correction if correction=Correction.HARTLAP or Correction.SELLENTIN
reduce_cov_factor=25,       # if = 1 Use standard covariance, even for the average
datafile=datafile,#"desi_kp4_abacus_cubicbox_pk_elg.pkl",
#datafile="desi_kp4_abacus_cubicbox_cv_pk_elg.pkl",   
data_location="/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles",
)

dataset_pk.set_realisation(None)


### read in the file with the best fit models for the mocks 

In [ ]:
directory = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/"
directory2 = "/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/"

#/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_abacus_cubic_ELG/output
#Barry_fit_CV_Pk_Recon.txt 
if CV: 
    file_fits_without_phi = pd.read_csv((directory + "desi_kp4_abacus_cubic_ELG/output/Barry_fit_Pk_Recon.txt"), header=0, nrows=52, index_col=False)#, skiprows=26)#, index_col=1)
    
    file_fits_with_phi = pd.read_csv((directory2 + "desi_kp4_abacus_cubic_ELG_phaseshiftfree/output/Barry_fit_Pk_Recon.txt"), header=0, nrows=52, index_col=False)#, nrows=26, skiprows=26)
    
    file_fits_without_phi = file_fits_without_phi[file_fits_without_phi['# Realisation'].apply(lambda x: 'CV' in x)]
    file_fits_with_phi = file_fits_with_phi[file_fits_with_phi['# Realisation'].apply(lambda x: 'CV' in x)]
    
else: 
    file_fits_without_phi = pd.read_csv((directory + "desi_kp4_abacus_cubic_ELG/output/Barry_fit_Pk_Recon.txt"), header=0, nrows=26, index_col=False)
    
    file_fits_with_phi = pd.read_csv((directory2 + "desi_kp4_abacus_cubic_ELG_phaseshiftfree/output/Barry_fit_Pk_Recon.txt"), header=0, nrows=26, index_col=False)
    
#params_model = file_fits_with_phi.to_dict()
#print(params_model)
    
#print(file_fits_without_phi[file_fits_without_phi['Realisation'] == '0'].to_dict())#[' alpha_par'][1])
#print(file_fits_with_phi)#file_fits_with_phi['# Realisation'] == '0'].to_dict())#[' alpha_par'][1])


params_model = file_fits_with_phi.to_dict()

params_model2 = file_fits_without_phi.to_dict()

#print(params_model)

### looking at the difference between models with and without phi as a free parameter - on average 

In [ ]:

average_diff_modelchoice_fits = np.zeros((len(dataset_pk.ks), 25)) 

average_models_with_phi = np.zeros((len(dataset_pk.ks), 25)) 
average_models_wo_phi = np.zeros((len(dataset_pk.ks), 25)) 

shift = 0
if CV:
    shift=25
for i in np.arange(shift,  25+shift):
                                         
    dataset_pk.set_realisation(i-shift)
    
    modelbest = PowerBeutler2017(
                    recon='sym',                   
                    isotropic=False,                          # Analytic marginalisation
                    n_poly=6,                                 # 6 polynomial terms for P(k)
                    vary_phase_shift_neff=True, 
                    use_classorcamb='CLASS',
                    marg='full', 
                    dilate_smooth=True
    )
    # print(params_model[' alpha_par'])
    sigma_nl_par = params_model[' alpha_par'][i+1]
    sigma_nl_perp = params_model[' alpha_perp'][i+1]
    sigma_s = params_model[' Sigma_s'][i+1]
    alpha_para = params_model[' Sigma_nl_par'][i+1]
    alpha_perp = params_model[' Sigma_nl_perp'][i+1]
    beta = params_model[' beta'][i+1]

    alpha, epsilon = modelbest.get_reverse_alphas(alpha_para, alpha_perp)

    modelbest.set_default("sigma_nl_par", sigma_nl_par)
    modelbest.set_default("sigma_nl_perp", sigma_nl_perp)
    modelbest.set_default("sigma_s", sigma_s)

    modelbest.set_default("alpha", alpha)
    modelbest.set_default("epsilon", epsilon)
    modelbest.set_default("beta_phase_shift", beta)
    
    modelbest.set_default("beta", 0.82)
    modelbest.set_default("b{0}_{1}", 0.51)
    
    cosmology = {
                "om": 0.3152,
                "h0": 0.6736,
                "z":  0.51,
                "ob": 0.0493,
                "ns": 0.9649,
                "mnu": 0.06,
                "reconsmoothscale": 15,
                "Neff": 3.046,
            }
                                         
    modelbest.set_cosmology(cosmology) # initialises self.camb object 
    modelbest.camb.get_data()
    ks = modelbest.camb.ks
    modelbest.set_data(dataset_pk.get_data())
    params_dict = {j.name: j.default for j in modelbest.params}
    
    pktemplate = np.loadtxt("../cosmodesi_KP4ELG_examplecode_make_picklefiles/DESI_Pk_template.dat")
    modelbest.kvals, modelbest.pksmooth, modelbest.pkratio = pktemplate.T

    new_chi_squared_best, dof, bband, modsbest, smooths_best = modelbest.get_model_summary(params_dict)
                                         
    average_models_with_phi[:,i-shift] = modsbest[0][0]
    
    modelbest1 = PowerBeutler2017(
                    recon='sym',                   
                    isotropic=False,                          # Analytic marginalisation
                    n_poly=6,                                 # 6 polynomial terms for P(k) 
                    use_classorcamb='CLASS',
                    marg='full', 
                    dilate_smooth=True
    )

    sigma_nl_par = params_model2[' alpha_par'][i+1]
    sigma_nl_perp = params_model2[' alpha_perp'][i+1]
    sigma_s = params_model2[' Sigma_s'][i+1]
    alpha_para = params_model2[' Sigma_nl_par'][i+1]
    alpha_perp = params_model2[' Sigma_nl_perp'][i+1]

    alpha, epsilon = modelbest1.get_reverse_alphas(alpha_para, alpha_perp)

    modelbest1.set_default("sigma_nl_par", sigma_nl_par)
    modelbest1.set_default("sigma_nl_perp", sigma_nl_perp)
    modelbest1.set_default("sigma_s", sigma_s)

    modelbest1.set_default("alpha", alpha)
    modelbest1.set_default("epsilon", epsilon)
    modelbest1.set_default("beta", 0.82)
    modelbest1.set_default("b{0}_{1}", 0.51)
    
                        
    modelbest1.set_cosmology(cosmology) # initialises self.camb object 
    modelbest1.camb.get_data()
    ks = modelbest1.camb.ks
    modelbest1.set_data(dataset_pk.get_data())
    params_dict = {j.name: j.default for j in modelbest1.params}
    
    pktemplate = np.loadtxt("../cosmodesi_KP4ELG_examplecode_make_picklefiles/DESI_Pk_template.dat")
    modelbest1.kvals, modelbest1.pksmooth, modelbest1.pkratio = pktemplate.T

    new_chi_squared_best1, dof, bband, modsbest1, smooths_best1 = modelbest1.get_model_summary(params_dict)
                                         
    
    average_diff_modelchoice_fits[:,i-shift] = modsbest1[0][0] - modsbest[0][0] 
                                         
    average_models_wo_phi[:,i-shift] = modsbest1[0][0]
    

average_diff_modelchoice_fits = np.mean(average_diff_modelchoice_fits, axis=1) 
average_models_wo_phi = np.mean(average_models_wo_phi, axis=1)
average_models_with_phi = np.mean(average_models_with_phi, axis=1)

                                         
plt.plot(dataset_pk.ks, average_diff_modelchoice_fits* dataset_pk.ks, label = r'$\langle diff \rangle$')
plt.plot(dataset_pk.ks, (average_models_wo_phi - average_models_with_phi) * dataset_pk.ks,
        label = r'$diff \langle \rangle$', linestyle='--')
dataset_pk.set_realisation(None)
lenk = len(dataset_pk.ks)
plt.errorbar(dataset_pk.ks, dataset_pk.data[:,0]-dataset_pk.data[:,0], np.sqrt(np.diag(dataset_pk.cov[0:lenk,0:lenk])) * dataset_pk.ks, linestyle='-.')
plt.ylabel(r"$k(\langle P(k)_i - P(k)_{i,\beta}) \rangle$ $\mathrm{Mpc}^2 h^{-2}$")
plt.xlabel(r"$k$ $\mathrm{Mpc} h^{-1}$")
suffix = ''
if CV:
    suffix = ' (CV)'
plt.title("Average difference for 25 Abacus Summit ELG mocks" + suffix)
plt.legend()
plt.show()                                        
                                         
